# Workflow submission notebook - Honeycomb structures

From this notebook we'll submit the configuration workflows for the honeycomb-restricted configurations of the Sn-doped structures of Li$_{1.2}$Mn$_{0.8}$O$_2$.

In [1]:
# Notebook header

import os
import numpy as np
from pybat.core import Cathode
from pymatgen import Composition, Element, Specie

In [2]:
# Set up the base cathode from with to generate the random honeycomb structures, based on the primitive Li2MnO3 cell.

honey_cat = Cathode.from_file("o3-abc-afm_Li2MnO3_prim.json")
honey_cat.remove_site_property("velocities")
honey_cat.replace(0, "Lr", properties={"magmom": 3.014}) # Placeholder element
honey_cat.replace(5, "Mn", properties={"magmom": 3.014}) # Stick to ferromagnetic ordering
honey_cat.sort()
honey_cat.make_supercell([2, 2, 2])

In [3]:
# Generate all configurations possible by substituting the placeholder element (Lr) by Li and Mn

Li_configurations = honey_cat.get_cation_configurations(
    substitution_sites=[i for i, s in enumerate(honey_cat) if s.specie == Element("Lr")],
    cation_list=["Li", "Mn"],
    sizes=[1,],
    concentration_restrictions={"Li": (0.30, 0.305)}, # This corresponds to about Li1.2Mn0.8O2 stoichiometry
    max_configurations=1e2
)

In [4]:
# Generate all possible Sn-doped structures for a specific choice of Sn-concentration

total_conf_list = []

# For each Li-Mn configuration, generate all Sn-Mn configurations
for conf in Li_configurations:
    total_conf_list.extend(conf.get_cation_configurations(
    substitution_sites=[i for i, s in enumerate(conf) if s.specie == Element("Mn")],
    cation_list=["Mn", "Sn"],
    sizes=[1,],
    concentration_restrictions={"Sn": (0.010, 0.012)}, # Sn-concentration is determined here
    max_configurations=None
))

In [5]:
len(total_conf_list)

47

---

In [6]:
from pybat.workflow.workflows import get_wf_configurations
from pybat.config import load_config

lpad = load_config("launchpad", "leibniz")
base_dir = "/scratch/antwerpen/202/vsc20248/batteries"

In [7]:
wf = get_wf_configurations(
    structure=honey_cat,
    directory=os.path.join(base_dir, str(total_conf_list[0].composition).replace(" ", "")),
    substitution_sites=[],
    sizes=[],
    configuration_list=total_conf_list,
    functional=("scanVdW", {}),
    in_custodian=True,
    number_nodes=4
)

In [8]:
lpad.add_wf(wf)

2020-01-22 10:22:52,477 INFO Added a workflow. id_map: {-1: 2}


{-1: 2}